In [ ]:
from os import listdir
from os.path import isfile, join
import re
import pandas as pd
from lexnlp.extract.en.entities import nltk_re
from lexnlp.nlp.en.segments import sentences
from lexnlp.utils import parse_df

In [ ]:
DATA_PATH = "..\data\interim\sfcr"

In [ ]:
txt_files = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f)) and f[-3:]=='txt']
documents = []
for file_name in txt_files:
    file = open(join(DATA_PATH, file_name), "rb")
    text = file.read().decode('utf-8')
    file.close()
    #text = text.replace("\n", " ")
    documents.append(text)
    
pickle_files = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f)) and f[-6:]=='pickle']
df = pd.DataFrame()
for file_name in pickle_files:
    df = df.append(pd.read_pickle(join(DATA_PATH, file_name)), ignore_index = True)

In [ ]:
print("Number of documents: " + str(len(documents)))
print("Number of sentences: " + str(len(df.index)))

In [ ]:
from sklearn.externals import joblib
import os

In [ ]:
from lexnlp.nlp.en.segments import titles
from dnbnlp.nlp.en.segments import solvency2_titles
from dnbnlp.nlp.en.segments import solvency2_document_year

In [ ]:
for idx in range(len(documents)):
    documents[idx] = documents[idx].replace('\uf0b7', "")
    documents[idx] = documents[idx].replace('\uf0a7', "")
    documents[idx] = documents[idx].replace('\uf0fc', "")
    documents[idx] = documents[idx].replace('\uf00c', "")
    documents[idx] = documents[idx].replace('\uf00d', "")
    documents[idx] = documents[idx].replace('\uf020', "")

## Build models voor title and document year locator

In [ ]:
solvency2_titles.build_model("..\\dnbnlp\\nlp\\en\\segments\\solvency2_titles_model.csv", DATA_PATH)
solvency2_document_year.build_model("..\\dnbnlp\\nlp\\en\\segments\\solvency2_document_year_model.csv", DATA_PATH)

## Reload title and document year locators

In [ ]:
solvency2_document_year.SECTION_SEGMENTER_MODEL = joblib.load(os.path.join(solvency2_document_year.MODULE_PATH, "./solvency2_document_year_locator.pickle"))
solvency2_titles.SECTION_SEGMENTER_MODEL = joblib.load(os.path.join(solvency2_titles.MODULE_PATH, "./solvency2_title_locator.pickle"))

In [ ]:
for item in range(len(documents)):
    print("Document           : " + str(txt_files[item]) + " : " + str(item))
    print("Predicted title    : " + str(list(solvency2_titles.get_titles(documents[item], score_threshold=0.5))))
    print("Predicted year     : " + str(list(solvency2_document_year.get_document_years(documents[item], score_threshold = 0.1))))    

In [ ]:
# feature_data = solvency2_document_year.build_document_document_year_features(documents[0])
# # Predict title lines
# predicted_lines = solvency2_document_year.SECTION_SEGMENTER_MODEL.predict_proba(feature_data)
# predicted_df = pd.DataFrame(predicted_lines, columns=["prob_false", "prob_true"])
# title_lines = predicted_df.loc[predicted_df["prob_true"] > 0.1, :].index.tolist()